<a href="https://colab.research.google.com/github/claireluo66/birds_dataset/blob/main/NoPretext_NOISE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torchvision.io import read_image, ImageReadMode

from operator import itemgetter
from itertools import groupby
from pathlib import Path
from tqdm.notebook import tqdm

import random
import numpy as np
from PIL import Image
import json
import os

In [2]:
!gdown --id 10X4KdsEsqhuZBID-iMAaZMmK6BYZXIVI
!tar -xf CUB_200_2011.tgz

Downloading...
From: https://drive.google.com/uc?id=10X4KdsEsqhuZBID-iMAaZMmK6BYZXIVI
To: /content/CUB_200_2011.tgz
1.15GB [00:05, 215MB/s]


In [9]:
from google.colab import files
uploaded = files.upload()

Saving train_test_val_split.txt to train_test_val_split.txt


In [10]:
def parse_datatype(item):
    for datatype in (int, float):
        try:
            return datatype(item)
        except ValueError:
            pass
    return item

def parse_and_filter(f, ids_to_load=None):
    for line in f:
        x = line.split()
        if ids_to_load is None or int(x[0]) in ids_to_load:
            yield [parse_datatype(i) for i in x]

In [68]:
import copy
class CUB(Dataset):
    def __init__(
        self,
        dataset_path,
        split_file_path="train_test_val_split.txt",
        *,
        type=0,
        images=True,
        labels=True,
        attributes=True,
        part_locs=False,
        mturk_part_locs=False,
        noise_mode=True,
        noise_ratio = 0.2,
        noise_file = "/content/drive/MyDrive/noise_file.json",
        transform=None,
        show_progress=True,
    ):

        root = Path(dataset_path)
        self.transform = transform

        with open(split_file_path) as f:
            self.ids = [id for id, type_ in parse_and_filter(f) if type_ == type]
            ids = set(self.ids)

        if show_progress:
            t = lambda x, desc: tqdm(x, total=len(ids), desc=desc)
        else:
            t = lambda x, _: x

        if images:
            #image paths
            with open(root / "images.txt") as f:
                self.image_paths = {
                    id: root / "images" / name
                    for id, name in t(parse_and_filter(f, ids), "images")
                }

        if labels:
            #image labels
            with open(root / "image_class_labels.txt") as f:
                self.labels = {id: label - 1 for id, label in t(parse_and_filter(f, ids), "labels")}


        if attributes:

            def issue(lines):
                for line in lines:
                    if len(line) > 5:
                        yield line[:4] + line[5:]
                    else:
                        yield line

            with open(root / "attributes" / "image_attribute_labels.txt") as f:
                self.attributes = {
                    id: torch.tensor([float(present) for _, _, present, _, _ in issue(lines)])
                    for id, lines in t(
                        groupby(parse_and_filter(f, ids), key=itemgetter(0)), "attributes"
                    )
                }


        if part_locs:
            with open(root / "parts" / "part_locs.txt") as f:
                self.part_locs = {
                    id: torch.tensor([info for _, *info in lines])
                    for id, lines in t(
                        groupby(parse_and_filter(f, ids), key=itemgetter(0)), "part_locs"
                    )
                }

        if mturk_part_locs:
            with open(root / "parts" / "part_click_locs.txt") as f:
                self.mturk_part_locs = {
                    id: torch.tensor([info for _, *info, _ in lines])
                    for id, lines in t(
                        groupby(parse_and_filter(f, ids), key=itemgetter(0)), "mturk_part_locs"
                    )
                }
       
        self.noise_ratio = noise_ratio
        if noise_mode:
            noise_labels = copy.deepcopy(self.labels)
            idx = list(self.labels.keys()) #11788 images
            random.shuffle(idx)
            num_noise = int(self.noise_ratio * len(idx))
            for i in idx[:num_noise]:
                noisy_label = random.randint(0,199) #200 classes
                noise_labels[i] = noisy_label
                
            print("save noisy labels to %s..."%noise_file)
            json.dump(noise_labels,open(noise_file,"w"))
            self.noise_labels = noise_labels


    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        id = self.ids[idx]
        item = {}

        if hasattr(self, "image_paths"):
            item["image"] = read_image(str(self.image_paths[id]), mode=ImageReadMode.RGB) / 255
        if self.transform:
                item["image"] = self.transform(item["image"])
        if hasattr(self, "labels"):
            item["label"] = self.labels[id]
        if hasattr(self, "attributes"):
            item["attributes"] = self.attributes[id]
        if hasattr(self, "attribute_classes"):
            item["attribute_classes"] = self.attribute_classes[id]
        if hasattr(self, "noise_labels"):
            item["noise_labels"] = self.noise_labels[id]

        return item

In [69]:
preprocess = {
    'train': transforms.Compose([
        transforms.Resize(256), 
        transforms.RandomRotation(45),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224), 
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [70]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
train_dataset = CUB("CUB_200_2011", type=0, transform=preprocess['train'], noise_file = "/content/drive/MyDrive/train_noise_file.json", noise_ratio=0.4)
test_dataset = CUB("CUB_200_2011", type=1, transform=preprocess['test'], noise_file = "/content/drive/MyDrive/test_noise_file.json", noise_ratio=0.4)
val_dataset = CUB("CUB_200_2011", type=2, transform=preprocess['test'], noise_file = "/content/drive/MyDrive/val_noise_file.json", noise_ratio=0.4)


save noisy labels to /content/drive/MyDrive/train_noise_file.json...



save noisy labels to /content/drive/MyDrive/test_noise_file.json...



save noisy labels to /content/drive/MyDrive/val_noise_file.json...


In [72]:
class MultimodalModel(nn.Module):
    def __init__(self):
        super().__init__()

        #IMAGES
        self.image_model = models.resnet50()

        #ATTRIBUTES
        self.attribute_model = nn.Sequential(
            nn.Linear(312, 250),
            nn.BatchNorm1d(250),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(250, 500),
            nn.BatchNorm1d(500),
            nn.ReLU(),
            nn.Dropout(0.3),
        )

        fusion_input_size = self.image_model.fc.out_features + 500
        self.fusion = nn.Sequential(
            nn.Linear(fusion_input_size, 500),
            nn.BatchNorm1d(500),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(500, 200),
        )

    def forward(self, imgs, attrs):
        image_out = self.image_model(imgs)
        attribute_out = self.attribute_model(attrs)

        combined = torch.cat([image_out, attribute_out], dim=1)
        return self.fusion(combined)

In [62]:
model = MultimodalModel()

In [73]:
def default_device():
    return torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    if isinstance(data, dict):
        return {k: to_device(v, device) for k, v in data.items()}
    return data.to(device)

class DeviceDataLoader():
    def __init__(self, dl, *, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        return len(self.dl)

device = default_device()
print(f"Using Device: {device}")

model.to(device)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
train_dataloader = DeviceDataLoader(train_dataloader, device=device)

test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)
test_dataloader = DeviceDataLoader(test_dataloader, device=device)

val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)
val_dataloader = DeviceDataLoader(val_dataloader, device=device)

Using Device: cuda:0


In [74]:
!pip install pkbar

In [75]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [78]:
from pkbar import Kbar

EPOCHS = 20

for epoch in range(EPOCHS):
    kbar = Kbar(target=len(train_dataloader) + len(val_dataloader), epoch=epoch, num_epochs=EPOCHS)

    # Train model

    model.train()
    for i, batch in enumerate(train_dataloader):
        optimizer.zero_grad()

        outputs = model(batch["image"], batch["attributes"])
        loss = criterion(outputs, batch["noise_labels"])
        loss.backward()
        optimizer.step()

        preds = F.softmax(outputs, dim=1).argmax(dim=1)
        acc = torch.sum(batch["noise_labels"] == preds) / len(preds)

        kbar.update(i, values=[("loss", loss.item()), ("acc", acc)])

    # Run validation

    model.eval()
    
    with torch.no_grad():
        for i, batch in enumerate(val_dataloader, start=len(train_dataloader)):
            outputs = model(batch["image"], batch["attributes"])
            loss = criterion(outputs, batch["label"])

            preds = F.softmax(outputs, dim=1).argmax(dim=1)
            acc = torch.sum(batch["label"] == preds) / len(preds)

            kbar.update(i, values=[("val_loss", loss.item()), ("val_acc", acc)])
    
    kbar.add(1)
    
print("Done")

Epoch: 1/20
157/157 [==============================] - 129s 820ms/step - loss: 5.3380 - acc: 0.0103 - val_loss: 5.3239 - val_acc: 0.0163
Epoch: 2/20
157/157 [==============================] - 131s 834ms/step - loss: 5.2349 - acc: 0.0153 - val_loss: 5.0609 - val_acc: 0.0430
Epoch: 3/20
157/157 [==============================] - 130s 826ms/step - loss: 5.0766 - acc: 0.0357 - val_loss: 4.7969 - val_acc: 0.0879
Epoch: 4/20
157/157 [==============================] - 127s 807ms/step - loss: 4.8968 - acc: 0.0664 - val_loss: 4.5620 - val_acc: 0.1378
Epoch: 5/20
157/157 [==============================] - 127s 808ms/step - loss: 4.7436 - acc: 0.0956 - val_loss: 4.3262 - val_acc: 0.1873
Epoch: 6/20
 22/157 [===>..........................] - ETA: 2:03 - loss: 4.5796 - acc: 0.1342

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe71fb90b90>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe71fb90b90>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

 32/157 [=====>........................] - ETA: 1:54 - loss: 4.5888 - acc: 0.1357

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe71fb90b90>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe71fb90b90>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

157/157 [==============================] - 127s 809ms/step - loss: 4.6271 - acc: 0.1257 - val_loss: 4.1966 - val_acc: 0.2168
Epoch: 7/20
157/157 [==============================] - 126s 804ms/step - loss: 4.5116 - acc: 0.1467 - val_loss: 4.0125 - val_acc: 0.2446
Epoch: 8/20
157/157 [==============================] - 125s 799ms/step - loss: 4.4116 - acc: 0.1673 - val_loss: 3.8796 - val_acc: 0.2706
Epoch: 9/20
157/157 [==============================] - 125s 799ms/step - loss: 4.3092 - acc: 0.1867 - val_loss: 3.7586 - val_acc: 0.2929
Epoch: 10/20
157/157 [==============================] - 127s 806ms/step - loss: 4.2189 - acc: 0.2053 - val_loss: 3.6482 - val_acc: 0.3017
Epoch: 11/20
157/157 [==============================] - 125s 798ms/step - loss: 4.1355 - acc: 0.2215 - val_loss: 3.5318 - val_acc: 0.3145
Epoch: 12/20
157/157 [==============================] - 126s 803ms/step - loss: 4.0542 - acc: 0.2395 - val_loss: 3.4803 - val_acc: 0.3435
Epoch: 13/20
157/157 [============================

In [79]:
def evaluate_model(dataloader=test_dataloader):
    correct = 0
    total = 0

    model.eval()

    with torch.no_grad():
        for batch in dataloader:
            outputs = model(batch["image"], batch["attributes"])
            preds = F.softmax(outputs, dim=1).argmax(dim=1)
            correct += torch.sum(batch["label"] == preds).item()
            total += len(preds)

    print(f"Test Accuracy: {correct}/{total} = {correct/total:.2%}")

In [80]:
evaluate_model()

Test Accuracy: 715/1773 = 40.33%
